In [ ]:

import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr  
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime
import plotly.plotly as py
import matplotlib.pyplot as plt
import json
import branca.colormap as cm
import folium
import seaborn as sns


## Carga de CSVs

## A continuacion se realiza la carga de archivos a los dataframe correspondientes.

In [ ]:
dfEstaciones =pd.read_csv('../CSVs/station.csv', thousands=',', low_memory=False)
dfClima=pd.read_csv('../CSVs/weather.csv', thousands=',', low_memory=False)
dfViajes = pd.read_csv('../CSVs/trip.csv', thousands=',', low_memory=False)
#Df creado por nuestro grupo con la informacion que queriamos analizar de Status.csv
dfTableID_DocksAvailable = pd.read_csv('../CSVs/tabla_id_dockAvailable.csv')

   # Filtrado de dfEstaciones

## El objetivo es simplemente ver la estructura del dataframe y saber que informacion nos puede brindar.

In [ ]:
print (dfEstaciones.shape) 

In [ ]:
dfEstaciones

In [ ]:
dfEstaciones.dtypes

In [ ]:
# No tenemos nullos
dfEstaciones.isnull().sum()

In [ ]:
#Pasamos el campo installation_date al tipo datetime
dfEstaciones.installation_date = pd.to_datetime(dfEstaciones.installation_date, format = "%m/%d/%Y").dt.date

#Probamos si funciona correctamente
print (dfEstaciones.installation_date.min())
print (dfEstaciones.installation_date.max())

# Filtrado de df Status.csv

## status.csv era un archivo de 2GB el cual fue reducido a una tabla con la informacion que ibamos a utilizar, esta contiene la estaciones agrupadas por ID junto con los docks_available en promedio. 
## El objetivo es obtener un porcentaje, el cual represanta la utilidad de esa estacion, es decir que tanto se utilizan sus docks en relacion a los disponibles.

In [ ]:
dfTableID_DocksAvailable.head()

In [ ]:
dfEstaciones.head()

In [ ]:
#Realizamos merge de dataframe 'dfEstaciones' con 'dfTableID_DocksAvailable' por ID de estacion
dfusagePercentagePerStation = dfEstaciones.merge(dfTableID_DocksAvailable, left_on='id', right_on='station_id').drop('station_id', 1)
dfusagePercentagePerStation

In [ ]:
#Le damos un nombre mas significado al feature
dfusagePercentagePerStation = dfusagePercentagePerStation.rename(columns={'docks_available': 'mean_of_docks_available'})
dfusagePercentagePerStation

In [ ]:
#Obtenemos el porcentaje de utilidad de cada estacion
dfusagePercentagePerStation['docks_usage_percentage'] = dfusagePercentagePerStation.mean_of_docks_available/dfusagePercentagePerStation.dock_count
dfusagePercentagePerStation

In [ ]:
dfusagePercentagePerStation.to_csv('../CSVs//usagePercentagePerStation.csv')

In [ ]:
dfusagePercentagePerStation.installation_date = pd.to_datetime(dfusagePercentagePerStation.installation_date, format = "%Y/%m/%d")


# Filtrado de dfClima
## El objetivo es simplemente ver la estructura del dataframe y saber que informacion nos puede brindar.

In [ ]:
dfClima

In [ ]:
dfClima.shape

In [ ]:
#Vemos que contiene muchos nulos
dfClima.isnull().sum()

In [ ]:
#Pasamos a datetime
dfClima.date = pd.to_datetime(dfClima.date, format='%m/%d/%Y')

In [ ]:
dfClima.head(10)

In [ ]:
dfClima.events.unique()

In [ ]:
#Limpio los datos bugeados, tenemos mucha informacion nula, la cual decidimos pasar a Normal
#Tambien corregi
dfClima.loc[dfClima.events == 'rain', 'events'] = "Rain"
dfClima.loc[dfClima.events.isnull(), 'events'] = "Normal"

In [ ]:
# EL zip_code es el codigo postal para las 5 ciudades de la bahia
dfClima.zip_code.unique()

In [ ]:
# Vemos cual de estos zip code tiene menos nulos
for zc in dfClima.zip_code.unique():
    print (dfClima[dfClima.zip_code == zc].isnull().sum())
    print ()

In [ ]:
#Trabajamos con el zip_code == 94107, es el mas 'limpio' de nulos, dada la cercania de todas las ciudades 
#la informacion no puede ser muy cambiante entre estas, por lo cual consideramos valido filtrar por una en particular.
dfClimaLimpio = dfClima[dfClima.zip_code == 94107]
dfClimaLimpio.shape

In [ ]:
dfClimaLimpio.events

In [ ]:
#Transformamos el feature events, en un nuevo dataframe el cual añadiremos a dfClimaLimpio
events = pd.get_dummies(dfClimaLimpio.events)
events

In [ ]:
#Mergeo los 2 data frames, eventos con dfClimaLimpio
dfClimaLimpio = dfClimaLimpio.merge(events, left_index = True, right_index = True)
#Saco los features que no voy a usar
dfClimaLimpio = dfClimaLimpio.drop(['events','zip_code'],1)

In [ ]:
#Todavia tenemos nulos
dfClimaLimpio.isnull().sum()

In [ ]:
#El campo max_gust_speed_mph no esta limpio todavia, lo trabajo
#Para cada valor de max_wind, busco la media de max_gust y la uso para rellenar los nulos.
dfClimaLimpio.loc[dfClimaLimpio.max_gust_speed_mph.isnull(), 'max_gust_speed_mph'] = \
                                dfClimaLimpio.groupby('max_wind_Speed_mph').\
                                max_gust_speed_mph.apply(lambda x: x.fillna(x.median()))

In [ ]:
# Listo para usar
dfClimaLimpio.isnull().sum()

In [ ]:
# el campo precipitation_inches lo deseamos de tipo numerico
dfClimaLimpio.dtypes

In [ ]:
dfClimaLimpio.precipitation_inches


In [ ]:
#Cambio el feature de string a numeric, los valores que son 'T' los cambio por NAN
dfClimaLimpio.precipitation_inches = pd.to_numeric(dfClimaLimpio.precipitation_inches, errors = 'coerce')

In [ ]:
#Change null values to the median, of values > 0, because T, I think, means True. 
#Therefore we want to find the median amount of precipitation on days when it rained.
dfClimaLimpio.loc[dfClimaLimpio.precipitation_inches.isnull(), 'precipitation_inches'] = \
dfClimaLimpio[dfClimaLimpio.precipitation_inches.notnull()].precipitation_inches.median()

## Cambiamos unidades norteamericanas por unidades mks

In [ ]:
# Cambiamos nombre de features
dfClimaLimpio=dfClimaLimpio.rename(columns={'max_temperature_f':'max_temperature_c', 'mean_temperature_f':'mean_temperature_c',\
            'max_dew_point_f':'max_dew_point_c','min_temperature_f':'min_temperature_c',\
            'mean_dew_point_f':'mean_dew_point_c','min_dew_point_f':'min_dew_point_c',\
            'max_sea_level_pressure_inches':'max_sea_level_pressure_cm','mean_sea_level_pressure_inches':'mean_sea_level_pressure_cm', \
            'min_sea_level_pressure_inches':'min_sea_level_pressure_cm','precipitation_inches':'precipitation_cm', \
            'max_visibility_miles':'max_visibility_km','mean_visibility_miles':'mean_visibility_km', \
            'min_visibility_miles':'min_visibility_km','max_wind_Speed_mph':'max_wind_Speed_kmh',\
            'mean_wind_speed_mph':'mean_wind_speed_kmh','max_gust_speed_mph':'max_gust_speed_kmh'})

In [ ]:
#Aplicamos las funciones para el cambio de unidades
def farToCelsius(x):
    x = (x - 32 )/ 1.8
    return float(x)

def pulgToCm(x):
    x = x * 2.54
    return float(x)
def milesToKm (x):
    x = x* 1.60934
    return float(x)

dfClimaLimpio.max_temperature_c = dfClimaLimpio.max_temperature_c.apply(farToCelsius)
dfClimaLimpio.mean_temperature_c = dfClimaLimpio.mean_temperature_c.apply(farToCelsius)
dfClimaLimpio.min_temperature_c =dfClimaLimpio.min_temperature_c.apply(farToCelsius)
dfClimaLimpio.max_dew_point_c = dfClimaLimpio.max_dew_point_c.apply(farToCelsius)
dfClimaLimpio.mean_dew_point_c = dfClimaLimpio.mean_dew_point_c.apply(farToCelsius)
dfClimaLimpio.min_dew_point_c = dfClimaLimpio.min_dew_point_c.apply(farToCelsius)
dfClimaLimpio.max_sea_level_pressure_cm = dfClimaLimpio.max_sea_level_pressure_cm.apply(pulgToCm)
dfClimaLimpio.mean_sea_level_pressure_cm= dfClimaLimpio.mean_sea_level_pressure_cm.apply(pulgToCm)
dfClimaLimpio.min_sea_level_pressure_cm = dfClimaLimpio.min_sea_level_pressure_cm.apply(pulgToCm)
dfClimaLimpio.precipitation_cm = dfClimaLimpio.precipitation_cm.apply(pulgToCm)
dfClimaLimpio.max_visibility_km = dfClimaLimpio.max_visibility_km.apply(milesToKm)
dfClimaLimpio.mean_visibility_km = dfClimaLimpio.mean_visibility_km.apply(milesToKm)
dfClimaLimpio.min_visibility_km = dfClimaLimpio.min_visibility_km.apply(milesToKm)
dfClimaLimpio.max_wind_Speed_kmh = dfClimaLimpio.max_wind_Speed_kmh.apply(milesToKm)
dfClimaLimpio.mean_wind_speed_kmh = dfClimaLimpio.mean_wind_speed_kmh.apply(milesToKm)
dfClimaLimpio.max_gust_speed_kmhh = dfClimaLimpio.max_gust_speed_kmh.apply(milesToKm)



dfClimaLimpio

# Filtrado de dfViajes
## El objetivo es simplemente ver la estructura del dataframe y saber que informacion nos puede brindar.

In [ ]:
dfViajes.isnull().sum()

In [ ]:
# El tamaño es manejable
dfViajes.shape

In [ ]:
dfViajes

In [ ]:
#Duracion de todos los viajes
dfViajes.duration.describe()

In [ ]:
# Lo paso a minutos y el promedio de viaje
dfViajes.duration /=60

In [ ]:
dfViajes.duration.describe()

Cantidad de viajes: 669959

In [ ]:
dfViajes.dtypes

In [ ]:
#Transformar de Object a DateTime
#El Date Time almacena YYYY-mm-dd
dfViajes.start_date = pd.to_datetime(dfViajes.start_date,format = '%m/%d/%Y %H:%M')
dfViajes.end_date = pd.to_datetime(dfViajes.end_date,format = '%m/%d/%Y %H:%M')

In [ ]:
dfViajes2 = dfViajes

# A partir de aqui comienza el analisis exploratorio de la informacion filtrada anteriormente.

## Objetivos: Analizar la distribucion de viajes en cada estacion, que tan usadas son las estaciones tanto de salida como de llegada. Observar la cantidad de viajes totales en cada hora del dia para saber cual es el horario donde mas usuarios usan las bicicletas. Duracion de los viajes en los distintos meses del año, 

In [ ]:
plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
#Primera vision para ver cuales de las estaciones fueron las mas usadas para la salida
#Podemos contar la cantidad de viajes solicitados en la Estacion mediante los datos del ID.
grupoStartStation = dfViajes.groupby(dfViajes['start_station_name']).count()
grupoStartStation.sort_values(by='id',ascending = False)

In [ ]:
#Primera vision para ver cuales de las estaciones fueron las mas usadas para la llegada
#Podemos contar la cantidad de viajes solicitados en la Estacion mediante los datos del ID.
grupoStartStation = dfViajes.groupby(dfViajes['end_station_name']).count()
grupoStartStation.sort_values(by='id',ascending = False)

In [ ]:
#23981 estaciones fueron simultaneamente estaciones de LLegada y Salida. 
#La mayoria de las estaciones fueron unicamente de Llegada o Salida
grupoEstacionesIguales = dfViajes[(dfViajes['start_station_name'] == (dfViajes['end_station_name']))]
grupoEstacionesIguales

In [ ]:
#Para los Viajes donde sus estaciones de llegada y salida son diferentes, estas son las estaciones de salida mas usadas.
grupoViajesDeEstacionesDiferentes = dfViajes[(dfViajes['start_station_name'] != (dfViajes['end_station_name']))]
ordenado = grupoViajesDeEstacionesDiferentes.groupby(dfViajes['start_station_name']).count().sort_values(by='id',ascending = False)
ordenado

In [ ]:
#Para los Viajes donde sus estaciones de llegada y salida son iguales, estas son las estaciones de salida mas usadas.
grupoViajesDeEstacionesIguales = dfViajes[(dfViajes['start_station_name'] == (dfViajes['end_station_name']))]
ordenado = grupoViajesDeEstacionesIguales.groupby(dfViajes['start_station_name']).count().sort_values(by='id',ascending = False)
ordenado

In [ ]:
#Estaciones de salida.Franja Horaria Desde las 06 hasta las 12
grupoSalidaManiana = dfViajes[(dfViajes.start_date.dt.hour >= 6 )&(dfViajes.start_date.dt.hour < 12)]
grupoSalidaManiana

In [ ]:
#Agrupo por Estacion y cuento la cantidad de ID de viaje en el grupo para saber cuantas veces fue usada una estacion salida
salida1 = grupoSalidaManiana.groupby(grupoSalidaManiana['start_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de salidas desde 06:00hs a 12:00hs')
plt.ylabel('numero de salidas')
plt.xlabel('estacion de salida')
salida1.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de salida.Franja Horaria Desde las 12 hasta las 14
grupoSalidaMediodia = dfViajes[(dfViajes.start_date.dt.hour >= 12 )&(dfViajes.start_date.dt.hour < 14)]
salida2 = grupoSalidaMediodia.groupby(grupoSalidaMediodia['start_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de salidas desde 12:00hs a 14:00hs')
plt.ylabel('numero de salidas')
plt.xlabel('estacion de salida')
salida2.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de salida.Franja Horaria Desde las 14  hasta las 19
grupoSalidaTarde = dfViajes[(dfViajes.start_date.dt.hour >= 14 )&(dfViajes.start_date.dt.hour < 19)]
salida3 = grupoSalidaTarde.groupby(grupoSalidaTarde['start_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de salidas desde 14:00hs a 19:00hs')
plt.ylabel('numero de salidas')
plt.xlabel('estacion de salida')
salida3.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de salida.Franja Horaria Desde las 19  hasta las 00
grupoSalidaNoche = dfViajes[(dfViajes.start_date.dt.hour >= 19)]
salida4 = grupoSalidaNoche.groupby(grupoSalidaNoche['start_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de salidas desde 19:00hs a 00:00hs')
plt.ylabel('numero de salidas')
plt.xlabel('estacion de salida')
salida4.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de salida.Franja Horaria Desde las 00 hasta las 6
grupoSalidaTrasnoche = dfViajes[(dfViajes.start_date.dt.hour < 6)]
salida5 = grupoSalidaTrasnoche.groupby(grupoSalidaTrasnoche['start_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de salidas desde 00:00hs a 6:00hs')
plt.ylabel('numero de salidas')
plt.xlabel('estacion de salida')
salida5.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de llegada.Franja Horaria Desde las 06 hasta las 12
grupoLlegadaManiana = dfViajes[(dfViajes.end_date.dt.hour >= 6 )&(dfViajes.end_date.dt.hour < 12)]
llegada1 = grupoLlegadaManiana.groupby(grupoLlegadaManiana['end_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de arribos desde 06:00hs a 12:00hs')
plt.ylabel('numero de arrivos')
plt.xlabel('estacion de llegada')
llegada1.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de llegada.Franja Horaria Desde las 12 hasta las 14
grupoLlegadaMediodia = dfViajes[(dfViajes.end_date.dt.hour >= 12 )&(dfViajes.end_date.dt.hour < 14)]
llegada2 = grupoLlegadaMediodia.groupby(grupoLlegadaMediodia['end_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de arribos desde 12:00hs a 14:00hs')
plt.ylabel('numero de arrivos')
plt.xlabel('estacion de llegada')
llegada2.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de llegada.Franja Horaria Desde las 14 hasta las 19
grupoLlegadaTarde = dfViajes[(dfViajes.end_date.dt.hour >= 14 )&(dfViajes.end_date.dt.hour < 19)]
llegada3 = grupoLlegadaTarde.groupby(grupoLlegadaTarde['end_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de arribos desde 14:00hs a 19:00hs')
plt.ylabel('numero de arribos')
plt.xlabel('estacion de llegada')
llegada3.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de salida.Franja Horaria Desde las 19 hasta las 00
grupoLlegadaNoche = dfViajes[(dfViajes.end_date.dt.hour >= 19)]
llegada4 = grupoLlegadaNoche.groupby(grupoLlegadaNoche['end_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de arribos desde 19:00hs a 00:00hs')
plt.ylabel('numero de arribos')
plt.xlabel('estacion de llegada')
llegada4.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
#Estaciones de llegada.Franja Horaria Desde las 00 hasta las 6
grupoLlegadaTrasnoche = dfViajes[(dfViajes.end_date.dt.hour < 6)]
llegada5 = grupoLlegadaTrasnoche.groupby(grupoLlegadaTrasnoche['end_station_name']).count().sort_values(by='id',ascending = False).id
plt.title('Cantidad de arribos desde 00:00hs a 06:00hs')
plt.ylabel('numero de arribos')
plt.xlabel('estacion de llegada')
llegada5.head(5).plot(kind = 'bar')
plt.show()

In [ ]:
dfViajes['hour'] = dfViajes.start_date.dt.hour
#averiguamos cuantos viajes se hicieron por hora
viajesHora = {}
for d in dfViajes.hour:
    if d not in viajesHora:
        viajesHora[d] = 1
    else:
        viajesHora[d] += 1
viajesHora

In [ ]:
#Grafico de la cantidad de viajes totales en funcion de las horas.
viajesPorHora = pd.DataFrame.from_dict(viajesHora, orient = 'index')
plt.title('Cantidad de Viajes por franja horaria')
plt.xlabel('horas del dia')
plt.xlim(0,23)
plt.ylabel('viajes totales')
indice = np.arange(24)
valores = ['0:00','1:00','2:00','3:00','4:00','5:00','6:00','7:00','8:00','9:00','10:00','11:00','12:00',
           '13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']
plt.xticks(indice,valores)
plt.plot(viajesPorHora)
plt.show()

In [ ]:
#PARA CADA MES CALCULAR LA DURACION PROMEDIO
import calendar
plt.ion()
meses = calendar.month_name[1:13]
indice = np.arange(12)
duracionPromedioMeses = np.arange(12,dtype=np.float)
dfViajes['month']= dfViajes.start_date.dt.month
for i in duracionPromedioMeses:
    dfViajesMes = dfViajes[dfViajes['month'] == (i + 1)]
    promedioMes = dfViajesMes.duration.mean()
    duracionPromedioMeses[i] = promedioMes
plt.title('Average length of travel according to months of the year')
plt.xlabel('Months')
plt.ylabel('Average duration (minutes)')
plt.xticks(indice,meses, size = 'small', color = 'b', rotation = 45)
plt.plot(duracionPromedioMeses)
plt.show()

In [ ]:
trace0 = go.Bar(
    x=llegada1.head(5),
    name='Cantidad de arribos desde 06:00hs a 12:00hs',
    )

trace1 = go.Bar(
    x=llegada2.head(5),
    name='Cantidad de arribos desde 12:00hs a 14:00hs'
     
)
trace2 = go.Bar(
    x=llegada3.head(5),
    name='Cantidad de arribos desde 14:00hs a 19:00hs'
     
)
trace3 = go.Bar(
    x=llegada4.head(5),
    name='Cantidad de arribos desde 19:00hs a 00:00hs'
     
)
trace4 = go.Bar(
    x=llegada5.head(5),
    name='Cantidad de arribos desde 00:00hs a 06:00hs'
     
)

data = [trace0, trace1,trace2,trace3,trace4]
layout = go.Layout(
    xaxis=dict(tickangle=0),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='angled-text-bar')

In [ ]:
trace0 = go.Bar(
    x=salida1.head(5),
    name='Cantidad de salidas desde 06:00hs a 12:00hs',
    )

trace1 = go.Bar(
    x=salida2.head(5),
    name='Cantidad de salidas desde 12:00hs a 14:00hs'
     
)
trace2 = go.Bar(
    x=salida3.head(5),
    name='Cantidad de salidas desde 14:00hs a 19:00hs'
     
)
trace3 = go.Bar(
    x=salida4.head(5),
    name='Cantidad de salidas desde 19:00hs a 00:00hs'
     
)
trace4 = go.Bar(
    x=salida5.head(5),
    name='Cantidad de salidas desde 00:00hs a 06:00hs'
     
)

data = [trace0, trace1,trace2,trace3,trace4]
layout = go.Layout(
    xaxis=dict(tickangle=0),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='angled-text-bar')


#  El objetivo de este nuevo analisis consiste en estudiar la variacion de los viajes con respecto a los dias de semana y a lo largo de un año, asi como tambien comprender como influyen las condiciones meteorologicas con respecto a los viajes en bicicleta. 

In [ ]:
dfViajes['date'] = dfViajes.start_date.dt.date
# Averiguamos cuantos viajes se hicieron por dia
dates = {}
for d in dfViajes.date:
    if d not in dates:
        dates[d] = 1
    else:
        dates[d] += 1

In [ ]:

dfCantViajes = pd.DataFrame.from_dict(dates, orient = "index")
dfCantViajes['date'] = dfCantViajes.index
dfCantViajes['viajes'] = dfCantViajes.ix[:,0]
dfSF_Bay = dfCantViajes.ix[:,1:3]
dfSF_Bay = dfSF_Bay.sort_values('date') #Ordeno por fecha
dfSF_Bay.reset_index(drop = True, inplace = True) # Queda mas prolijo con indice de 0 a ...
dfSF_Bay

In [ ]:
data = [go.Scatter( x=dfSF_Bay['date'], y=dfSF_Bay['viajes'] )]

layout = dict(
    title='Distribucion de viajes en linea de tiempo',
    yaxis=dict(title='Viajes')
           
    )
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Time Series with Rangeslider")

In [ ]:
data = [go.Scatter(
          x=dfSF_Bay.date,
          y=dfSF_Bay.viajes,
            mode='markers',
    showlegend=False)
           ]
layout = dict(
    title='Distribucion de viajes en linea de tiempo',
    yaxis=dict(title='Viajes')
           
    )
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Time Series with Rangeslider")

In [ ]:
print((dfCantViajes).info()) 

In [ ]:
print((dfViajes).info()) 


In [ ]:
#Hacemos un merge, para obtener los viajes con el clima
dfSF_Bay = dfSF_Bay.merge(dfClimaLimpio, on = dfSF_Bay.date)
dfSF_Bay

In [ ]:
dfSF_Bay['date'] = dfSF_Bay['date_x']
dfSF_Bay.drop(['date_y','date_x'],1, inplace= True)
#Limpio esos features repetidos

In [ ]:
dfSF_Bay #todo ok

In [ ]:
#Buscamos los feriados en el calendario estadounidense
calendar = USFederalHolidayCalendar()
holidays = calendar.holidays(start=dfSF_Bay.date.min(), end=dfSF_Bay.date.max())
holidays

In [ ]:
#Buscamos los dias laborables en el calendario estadounidense
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
business_days = pd.DatetimeIndex(start=dfSF_Bay.date.min(), end=dfSF_Bay.date.max(), freq=us_bd)

In [ ]:
business_days = pd.to_datetime(business_days, format='%Y/%m/%d').date
holidays = pd.to_datetime(holidays, format='%Y/%m/%d').date

In [ ]:
# Lo agregamos al dataframe
dfSF_Bay['business_day'] = dfSF_Bay.date.isin(business_days)
dfSF_Bay['holiday'] = dfSF_Bay.date.isin(holidays)
dfSF_Bay

In [ ]:
dfSF_Bay.dtypes

In [ ]:
#Convert True to 1 and False to 0
dfSF_Bay.business_day = dfSF_Bay.business_day.map(lambda x: 1 if x == True else 0)
dfSF_Bay.holiday = dfSF_Bay.holiday.map(lambda x: 1 if x == True else 0)
dfSF_Bay

In [ ]:
#Convert date to the important features, year, month, weekday (0 = Monday, 1 = Tuesday...)
#We don't need day because what it represents changes every year.
dfSF_Bay['year'] = pd.to_datetime(dfSF_Bay['date']).dt.year
dfSF_Bay['month'] = pd.to_datetime(dfSF_Bay['date']).dt.month
dfSF_Bay['weekday'] = pd.to_datetime(dfSF_Bay['date']).dt.weekday
dfSF_Bay

In [ ]:
#Lo usamos para grafico Meses vs Viajes
meses = dfSF_Bay.groupby('month').sum()
#Lo usamos para grafico Dias vs Viajes
dia = dfSF_Bay.groupby('weekday').sum()


In [ ]:
data = [go.Scatter(
          x=['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre'],
          y=meses.viajes)]
layout = dict(
    title='Cantidad de viajes por mes',
    xaxis=dict(title='Meses'),
    yaxis=dict(title='Viajes') 
    )
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Time Series with Rangeslider")


In [ ]:
x1=['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo']
data= [go.Scatter(
          x=x1,
          y=dia.viajes)]

layout = dict(
    title='Cantidad de viajes por mes',
    xaxis=dict(title='Dia'),
    yaxis=dict(title='Viajes')
       
      
    )


fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename = "Time Series with Rangeslider")

### DFTIMELINE

In [ ]:
from collections import defaultdict
salida = {}
entrada ={}
epa = []
for d in dfViajes.end_station_name:
    if d not in salida:
        salida[d] =1
    else:
        salida[d] +=1
        
for d in dfViajes.start_station_name:
    if d not in entrada:
        entrada[d] =1
    else:
        entrada[d] +=1  
        
        
a=salida.items()
b=entrada.items()
a.extend(b)
d = defaultdict(list)
for k, v in a:
    d[k].append(v)
    

In [ ]:
dfRanking = pd.DataFrame.from_dict(d, orient="index")
dfRanking['name'] = dfRanking.index
dfRanking['bicicletas_que_salieron']= dfRanking.ix[:,0].astype(float)
dfRanking['bicicletas_que_entraron']= dfRanking.ix[:,1].astype(float)
dfRanking['promedio']= (dfRanking.bicicletas_que_entraron + dfRanking.bicicletas_que_salieron)/2

dfRanking.reset_index(drop=True, inplace=True)
#dfRanking.drop([0,1],1)

del dfRanking[0]
del dfRanking[1]

In [ ]:
dfTimeline = pd.DataFrame(columns=['name','Inauguracion','Uso_de_Estacion_P','Salida_P','Entrada_P'])
dfTimeline.name = dfEstaciones.name
dfTimeline.Inauguracion = dfEstaciones.installation_date
dfTimeline = pd.merge(dfTimeline, dfRanking, on='name', how='inner')
dfTimeline.Inauguracion = pd.to_datetime(dfTimeline.Inauguracion, format = "%Y/%m/%d")

dfTimeline = pd.merge(dfTimeline, dfEstaciones, on='name', how='inner')
del dfTimeline['installation_date']


In [ ]:
bicicletasQueSalieron = dfTimeline.bicicletas_que_salieron.sum()
bicicletasQueEntraron = dfTimeline.bicicletas_que_entraron.sum()
bicicletasPromedio = dfTimeline.promedio.sum()

dfTimeline.Uso_de_Estacion_P = dfTimeline.promedio.apply(lambda x: (100*x)/bicicletasPromedio)
dfTimeline.Salida_P = dfTimeline.bicicletas_que_salieron.apply(lambda x: float((100*x)/bicicletasQueSalieron))
dfTimeline.Entrada_P = dfTimeline.bicicletas_que_entraron.apply(lambda x: float((100*x)/bicicletasQueEntraron))


In [ ]:
dfTimeline.sort_values(by='promedio', ascending=False)


In [ ]:
import plotly 
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.graph_objs import *
plotly.offline.init_notebook_mode()
plotly.tools.set_credentials_file(username='AARdfSF_Bay', api_key='XFLHonutbTpz1cZ77dcl')



In [ ]:
dfTimeline = dfTimeline.sort_values(by= 'Uso_de_Estacion_P')
data = [
    go.Bar(
        x=dfTimeline.Uso_de_Estacion_P, # assign x as the dataframe column 'x'
        y=dfTimeline.name,
        orientation='h',
        
    )
]
layout = dict(
    title='Ranking de estaciones por uso',
    xaxis=dict(title='Porcentaje con respecto al total de estaciones'),     
    )

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-bar')


In [ ]:
dfTimeline = dfTimeline.sort_values(by= 'Salida_P', ascending=False)
data =  [go.Bar(
        x=dfTimeline.Salida_P.head(10), # assign x as the dataframe column 'x'
        y=dfTimeline.name,
        orientation='h',
        )]
    

layout = dict(
    title='Top 10 estaciones por promedio de salida',
    xaxis=dict(title='Porcentaje con respecto al total de estaciones'),     
    )

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-bar')



In [ ]:
dfTimeline = dfTimeline.sort_values(by= 'Entrada_P', ascending=False)
data =  [go.Bar(
        x=dfTimeline.Entrada_P.head(10), # assign x as the dataframe column 'x'
        y=dfTimeline.name,
        orientation='h',
        )]
    

layout = dict(
    title='Top 10 estaciones por promedio de entrada',
    xaxis=dict(title='Porcentaje con respecto al total de estaciones'),     
    )

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-bar')


In [ ]:

cantidades = dfTimeline.groupby('city', as_index=False).count()
cantidades = cantidades['name']
trace1 = go.Bar(
    x=['Mountain View', 'Palo Alto', 'Redwood City', 'San Francisco', 'San Jose'],
    y=cantidades.values.tolist(),

)
data = [trace1]
layout = dict(
    title='Cantidad de estaciones por ciudad',
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='stacked-bar')




In [ ]:
dfSF_BayHeatmap2 =dfSF_Bay.loc[:,ejeY].corr().head(1)
fig, ax = plt.subplots(figsize=(11,6)); 
df =dfSF_BayHeatmap2.drop(['viajes'],1)
ax = plt.axes()
cg=sns.heatmap(df,cmap='RdBu_r', ax = ax)
ax.set_title('Correlacion cantidad de viajes con parametros')
for item in cg.get_xticklabels():
    item.set_rotation(90)
plt.gcf().subplots_adjust(bottom=0.6)


In [ ]:
dfTimeline = dfTimeline.sort_values('Inauguracion')
data = [
    go.Scatter(
        x=dfTimeline.id.count(), 
        y=dfTimeline.Inauguracion
    )
]
layout = dict(
    title='Creacion de estaciones a lo largo del tiempo',
    xaxis=dict(title='Cantidad de estaciones'),     
    )
fig = go.Figure(data=data, layout=layout)
url = plotly.offline.iplot(fig, filename='pandas-horizontal-bar')



# Finalmente se propone un analisis desde la perspectiva geolocalizada, estudiar las variables analizadas anteriormente en la distribucion de un mapa de diversas formas.

In [ ]:
latPromedio = dfEstaciones.lat.mean()
longPromedio = dfEstaciones.long.mean()
cantidadTotalDeBicicletasSoportadas = dfEstaciones.dock_count.sum()
print cantidadTotalDeBicicletasSoportadas

In [ ]:
map1 = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
map1Copy = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
map2 = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
startStationMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
viajesEntreEstacionesMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")

In [ ]:
grupoDeUbicaciones = folium.FeatureGroup(name='Ubicación de estaciones 1')
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    grupoDeUbicaciones.add_child(folium.Marker(location=[lat,lon], popup=nombre))
    
map1.add_child(grupoDeUbicaciones)

In [ ]:
# Ahora intento version con MarkerCluster
# map1Copy.add_child(folium.MarkerCluster(locations=list(zip(dfEstaciones.long, dfEstaciones.lat))))

clusterDeUbicaciones = folium.MarkerCluster().add_to(map1Copy)
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicaciones)
    
# map1.add_child(grupoDeUbicaciones)
map1Copy

In [ ]:
grupoDeUbicacionesConCantidades = folium.FeatureGroup(name='Ubicación de estaciones')
for lat, lon, nombre, cantidadDeBicicletasSoportadas in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name, dfEstaciones.dock_count):
    grupoDeUbicacionesConCantidades.add_child(folium.CircleMarker(location=[lat,lon], popup=nombre, 
                                                     radius=cantidadDeBicicletasSoportadas, 
                                                     fill_color='#b3ffb3', color='#00ff00'))
    
map2.add_child(grupoDeUbicacionesConCantidades)

In [ ]:
dfStartStationCount = dfViajes[['id', 'start_station_id']].groupby(by='start_station_id', as_index=False)\
                                                .count().rename(columns={'id': 'cantidad_de_partidas'})
dfStartStationCount

In [ ]:
dfPartidas = dfEstaciones[['id', 'lat', 'long', 'name']].merge(pd.DataFrame(dfStartStationCount), \
         left_on='id', right_on='start_station_id', how='right')
dfPartidas

In [ ]:
cantidad_de_partidas_maxima = dfPartidas.cantidad_de_partidas.max()
cantidad_de_partidas_maxima

In [ ]:
grupoDeUbicacionesConCantidadesDePartidas = folium.FeatureGroup(name='Ubicación de estaciones con infor de #partidas')
for lat, lon, nombre, cant in zip(dfPartidas.lat, dfPartidas.long, dfPartidas.name, dfPartidas.cantidad_de_partidas):
    grupoDeUbicacionesConCantidadesDePartidas.add_child(folium.CircleMarker(location=[lat,lon], popup=nombre, 
                                                     radius=cant/1000,
                                                     fill_color='#8A2BE2', color='#00ff00'))
    
startStationMap.add_child(grupoDeUbicacionesConCantidadesDePartidas)

In [ ]:
dfViajesEntreEstaciones = dfViajes[['id', 'start_station_id', 'end_station_id']]\
        .groupby(by=['start_station_id', 'end_station_id'], as_index=False)\
        .count().rename(columns={'id': 'cantidad'})
dfViajesEntreEstaciones

In [ ]:
dfViajesEntreEstaciones = dfViajesEntreEstaciones.merge(right=dfEstaciones[['lat', 'long', 'id', 'name', 'city']], \
                              left_on='start_station_id', right_on='id')\
                            .drop(['id'], 1).rename(columns={'lat': 'start_station_lat', 'long': \
                                                          'start_station_long', 'name': 'start_station_name', \
                                                             'city': 'start_city'})

In [ ]:
dfViajesEntreEstaciones = dfViajesEntreEstaciones.merge(right=dfEstaciones[['lat', 'long', 'id', 'name', 'city']], \
                              left_on='end_station_id', right_on='id')\
                            .drop(['id'], 1).rename(columns={'lat': 'end_station_lat', 'long': \
                                                          'end_station_long', 'name': 'end_station_name', \
                                                             'city': 'end_city'})
dfViajesEntreEstaciones.sort_values(by='cantidad', ascending=True)

In [ ]:
dfViajesEntreEstaciones.cantidad.describe()

## Analizo distribucion de datos


In [ ]:
dfViajesEntreEstaciones.filter(items=['start_station_id', 'cantidad']).hist('cantidad', bins=4)

In [ ]:
maxCantidadDeViajes = dfViajesEntreEstaciones.cantidad.max()
promedioCantidadDeViajes = dfViajesEntreEstaciones.cantidad.mean()
dfViajesEntreEstaciones.cantidad.max()

In [ ]:
# colores = cm.LinearColormap(['green', 'orange', 'red'], vmax=maxCantidadDeViajes, 
#                   index=[0, promedioCantidadDeViajes, maxCantidadDeViajes])

# colores = cm.LinearColormap(['red', 'orange', 'yellow', 'green'], vmax = maxCantidadDeViajes)

colores = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajes, index=[1, 1500, 3100, 4650, maxCantidadDeViajes],
                       caption='step')
colores.caption = 'Cantidad de viajes entre estaciones'
# colores = cm.linear.Set1.scale(0, maxCantidadDeViajes).to_step(
# #     n=4,
#     index=[1, 71.5, 191, 440, 6216],
# #     method='quantiles',
# #     round_method='int'
# )
# colores.to_linear()
colores2 = cm.linear.YlGn.scale(0, 6000)
#colores3 = cm.linear.RdGy.to_step(100).scale(0, 6000)


colores

In [ ]:
linesMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
linesMap.add_child(colores)

In [ ]:
clusterDeUbicaciones = folium.MarkerCluster().add_to(linesMap)
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicaciones)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstaciones.start_station_lat,\
                                                                       dfViajesEntreEstaciones.start_station_long,\
                                                                       dfViajesEntreEstaciones.start_station_name,\
                                                                       dfViajesEntreEstaciones.end_station_lat,\
                                                                       dfViajesEntreEstaciones.end_station_long,\
                                                                       dfViajesEntreEstaciones.end_station_name,\
                                                                       dfViajesEntreEstaciones.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=colores(cant), weight=3, opacity=1).add_to(linesMap)
#     linesMap.add_child(c)
    
# startStationMap.add_child(grupoDeViajesEntreEstaciones)
linesMap

In [ ]:
dfViajesEntreEstaciones.start_city.unique()

# Mapa con viajes en San Jose

In [ ]:
dfViajesEntreEstacionesEnSanJose = (dfViajesEntreEstaciones[dfViajesEntreEstaciones.start_city == 'San Jose']).drop('start_city', 1)
dfViajesEntreEstacionesEnSanJose = (dfViajesEntreEstacionesEnSanJose[dfViajesEntreEstacionesEnSanJose.end_city == 'San Jose']).drop('end_city', 1)
dfViajesEntreEstacionesEnSanJose

In [ ]:
latSanJosePromedio = dfViajesEntreEstacionesEnSanJose.start_station_lat.mean()
longSanJosePromedio = dfViajesEntreEstacionesEnSanJose.start_station_long.mean()

In [ ]:
viajesEntreEstacionesEnSanJoseMap = folium.Map(location=[latSanJosePromedio, longSanJosePromedio], tiles = "Stamen Terrain", zoom_start=14)
viajesEntreEstacionesEnSanJoseMap

In [ ]:
dfViajesEntreEstacionesEnSanJose.cantidad.describe()

In [ ]:
maxCantidadDeViajesEnSanJose = dfViajesEntreEstacionesEnSanJose.cantidad.max()
# promedioCantidadDeViajesEnSanJose = dfViajesEntreEstacionesEnSanJose.cantidad.mean()

In [ ]:
coloresSanJose = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnSanJose, index=[3, 32, 65, 135, maxCantidadDeViajesEnSanJose],
                       caption='step')
coloresSanJose.caption = 'Cantidad de viajes entre estaciones en San Jose'
coloresSanJose
coloresSanJose2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnSanJose)

In [ ]:
viajesEntreEstacionesEnSanJoseMap.add_child(coloresSanJose)

In [ ]:
clusterDeUbicacionesSanJose = folium.MarkerCluster().add_to(viajesEntreEstacionesEnSanJoseMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnSanJose.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnSanJose.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnSanJose.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesSanJose)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnSanJose.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanJose.start_station_long,\
                                                                       dfViajesEntreEstacionesEnSanJose.start_station_name,\
                                                                       dfViajesEntreEstacionesEnSanJose.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanJose.end_station_long,\
                                                                       dfViajesEntreEstacionesEnSanJose.end_station_name,\
                                                                       dfViajesEntreEstacionesEnSanJose.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresSanJose(cant), weight=3, opacity=1).add_to(viajesEntreEstacionesEnSanJoseMap)
viajesEntreEstacionesEnSanJoseMap

## Grafico con n estaciones mas activas


In [ ]:
n = 20

In [ ]:
cotaInferiorSanJose = dfViajesEntreEstacionesEnSanJose.cantidad.nlargest (n).min()

In [ ]:
dfViajesEntreEstacionesEnSanJoseTopN = dfViajesEntreEstacionesEnSanJose[dfViajesEntreEstacionesEnSanJose['cantidad'] \
                                                                        >= cotaInferiorSanJose]
dfViajesEntreEstacionesEnSanJoseTopN

In [ ]:
topViajesEntreEstacionesEnSanJoseMap = folium.Map(location=[latSanJosePromedio, longSanJosePromedio], tiles = "Stamen Terrain", zoom_start=14)
topViajesEntreEstacionesEnSanJoseMap

In [ ]:
dfViajesEntreEstacionesEnSanJoseTopN.cantidad.describe()

In [ ]:
coloresSanJoseTopN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnSanJose, index=[438, 552, 666, 857, maxCantidadDeViajesEnSanJose],
                       caption='step')
coloresSanJoseTopN.caption = 'Cantidad de viajes entre estaciones en San Jose'

coloresSanJoseTopN2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnSanJose)

coloresSanJoseTopN

In [ ]:
topViajesEntreEstacionesEnSanJoseMap.add_child(coloresSanJoseTopN)

In [ ]:
clusterDeUbicacionesSanJose = folium.MarkerCluster().add_to(topViajesEntreEstacionesEnSanJoseMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnSanJose.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnSanJose.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnSanJose.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesSanJose)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnSanJoseTopN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanJoseTopN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnSanJoseTopN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnSanJoseTopN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanJoseTopN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnSanJoseTopN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnSanJoseTopN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresSanJoseTopN(cant), weight=5, opacity=1).add_to(topViajesEntreEstacionesEnSanJoseMap)
topViajesEntreEstacionesEnSanJoseMap

## Grafico con n estaciones menos activas


In [ ]:
n = 20

In [ ]:
cotaSuperiorSanJose = dfViajesEntreEstacionesEnSanJose.cantidad.nsmallest(n).max()

In [ ]:
dfViajesEntreEstacionesEnSanJoseBottomN = dfViajesEntreEstacionesEnSanJose[dfViajesEntreEstacionesEnSanJose['cantidad'] \
                                                                        <= cotaSuperiorSanJose]
dfViajesEntreEstacionesEnSanJoseBottomN

In [ ]:
bottomViajesEntreEstacionesEnSanJoseMap = folium.Map(location=[latSanJosePromedio, longSanJosePromedio], tiles = "Stamen Terrain", zoom_start=14)
bottomViajesEntreEstacionesEnSanJoseMap

In [ ]:
dfViajesEntreEstacionesEnSanJoseBottomN.cantidad.describe()

In [ ]:
coloresSanJoseBottomN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=13, index=[3, 5, 8, 10, 13],
                       caption='step')
coloresSanJoseBottomN.caption = 'Cantidad de viajes entre estaciones con menos viajes entre ellas en San Jose'

coloresSanJoseBottomN2 = cm.linear.YlGn.scale(0, 13)

coloresSanJoseBottomN

In [ ]:
bottomViajesEntreEstacionesEnSanJoseMap.add_child(coloresSanJoseBottomN)

In [ ]:
clusterDeUbicacionesSanJose = folium.MarkerCluster().add_to(bottomViajesEntreEstacionesEnSanJoseMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnSanJose.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnSanJose.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnSanJose.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesSanJose)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnSanJoseBottomN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanJoseBottomN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnSanJoseBottomN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnSanJoseBottomN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanJoseBottomN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnSanJoseBottomN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnSanJoseBottomN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresSanJoseBottomN(cant), weight=5, opacity=1).add_to(bottomViajesEntreEstacionesEnSanJoseMap)
bottomViajesEntreEstacionesEnSanJoseMap

# Mapa con viajes en Mountain View

In [ ]:
dfViajesEntreEstacionesEnMountainView = (dfViajesEntreEstaciones[dfViajesEntreEstaciones.start_city == 'Mountain View']).drop('start_city', 1)
dfViajesEntreEstacionesEnMountainView = (dfViajesEntreEstacionesEnMountainView[dfViajesEntreEstacionesEnMountainView.end_city == 'Mountain View']).drop('end_city', 1)
dfViajesEntreEstacionesEnMountainView

In [ ]:
latMountainViewPromedio = dfViajesEntreEstacionesEnMountainView.start_station_lat.mean()
longMountainViewPromedio = dfViajesEntreEstacionesEnMountainView.start_station_long.mean()

In [ ]:
viajesEntreEstacionesEnMountainViewMap = folium.Map(location=[latMountainViewPromedio, longMountainViewPromedio], tiles = "Stamen Terrain", zoom_start=14)
viajesEntreEstacionesEnMountainViewMap

In [ ]:
dfViajesEntreEstacionesEnMountainView.cantidad.describe()

In [ ]:
maxCantidadDeViajesEnMountainView = dfViajesEntreEstacionesEnMountainView.cantidad.max()

In [ ]:
coloresMountainView = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnMountainView, index=[2, 30, 101, 280, maxCantidadDeViajesEnMountainView],
                       caption='step')
coloresMountainView.caption = 'Cantidad de viajes entre estaciones en Mountain View'
coloresMountainView
coloresSanJose2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnMountainView)

In [ ]:
viajesEntreEstacionesEnMountainViewMap.add_child(coloresMountainView)

In [ ]:
clusterDeUbicacionesMountainView = folium.MarkerCluster().add_to(viajesEntreEstacionesEnMountainViewMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnMountainView.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnMountainView.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnMountainView.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesMountainView)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnMountainView.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnMountainView.start_station_long,\
                                                                       dfViajesEntreEstacionesEnMountainView.start_station_name,\
                                                                       dfViajesEntreEstacionesEnMountainView.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnMountainView.end_station_long,\
                                                                       dfViajesEntreEstacionesEnMountainView.end_station_name,\
                                                                       dfViajesEntreEstacionesEnMountainView.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresMountainView(cant), weight=3, opacity=1).add_to(viajesEntreEstacionesEnMountainViewMap)
viajesEntreEstacionesEnMountainViewMap

### Grafico con n estaciones menos activas

In [ ]:
n = 20

In [ ]:
cotaSuperiorMountainView = dfViajesEntreEstacionesEnMountainView.cantidad.nsmallest(n).max()

In [ ]:
dfViajesEntreEstacionesEnMountainViewBottomN = dfViajesEntreEstacionesEnMountainView[dfViajesEntreEstacionesEnMountainView['cantidad'] \
                                                                        <= cotaSuperiorMountainView]
dfViajesEntreEstacionesEnMountainViewBottomN

In [ ]:
bottomViajesEntreEstacionesEnMountainViewMap = folium.Map(location=[latMountainViewPromedio, longMountainViewPromedio], tiles = "Stamen Terrain", zoom_start=14)
bottomViajesEntreEstacionesEnMountainViewMap

In [ ]:
dfViajesEntreEstacionesEnMountainViewBottomN.cantidad.describe()

In [ ]:
coloresMountainViewBottomN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=67, index=[2, 8, 21, 45, 67],
                       caption='step')
coloresMountainViewBottomN.caption = 'Cantidad de viajes entre estaciones con menos viajes entre ellas en Mountain View'

coloresMountainViewBottomN2 = cm.linear.YlGn.scale(0, 67)

coloresMountainViewBottomN

In [ ]:
bottomViajesEntreEstacionesEnMountainViewMap.add_child(coloresMountainViewBottomN)

In [ ]:
clusterDeUbicacionesMountainView = folium.MarkerCluster().add_to(bottomViajesEntreEstacionesEnMountainViewMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnMountainView.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnMountainView.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnMountainView.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesMountainView)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnMountainViewBottomN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnMountainViewBottomN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnMountainViewBottomN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnMountainViewBottomN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnMountainViewBottomN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnMountainViewBottomN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnMountainViewBottomN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresMountainViewBottomN(cant), weight=5, opacity=1).add_to(bottomViajesEntreEstacionesEnMountainViewMap)
bottomViajesEntreEstacionesEnMountainViewMap

# Mapa con viajes en San Francisco

In [ ]:
dfViajesEntreEstacionesEnSanFrancisco = (dfViajesEntreEstaciones[dfViajesEntreEstaciones.start_city == 'San Francisco']).drop('start_city', 1)
dfViajesEntreEstacionesEnSanFrancisco = (dfViajesEntreEstacionesEnSanFrancisco[dfViajesEntreEstacionesEnSanFrancisco.end_city == 'San Francisco']).drop('end_city', 1)
dfViajesEntreEstacionesEnSanFrancisco

In [ ]:
latSanFranciscoPromedio = dfViajesEntreEstacionesEnSanFrancisco.start_station_lat.mean()
longSanFranciscoPromedio = dfViajesEntreEstacionesEnSanFrancisco.start_station_long.mean()

In [ ]:
viajesEntreEstacionesEnSanFranciscoMap = folium.Map(location=[latSanFranciscoPromedio, longSanFranciscoPromedio], tiles = "Stamen Terrain", zoom_start=14)
viajesEntreEstacionesEnSanFranciscoMap

In [ ]:
dfViajesEntreEstacionesEnSanFrancisco.cantidad.describe()

In [ ]:
maxCantidadDeViajesEnSanFrancisco = dfViajesEntreEstacionesEnSanFrancisco.cantidad.max()

In [ ]:
coloresSanFrancisco = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnSanFrancisco, index=[9, 149, 284, 565, maxCantidadDeViajesEnSanFrancisco],
                       caption='step')
coloresSanFrancisco.caption = 'Cantidad de viajes entre estaciones en San Francisco'
coloresSanFrancisco
coloresSanFrancisco2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnSanFrancisco)

In [ ]:
viajesEntreEstacionesEnSanFranciscoMap.add_child(coloresSanFrancisco)

In [ ]:
clusterDeUbicacionesSanFrancisco = folium.MarkerCluster().add_to(viajesEntreEstacionesEnSanFranciscoMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnSanFrancisco.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnSanFrancisco.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnSanFrancisco.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesSanFrancisco)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnSanFrancisco.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanFrancisco.start_station_long,\
                                                                       dfViajesEntreEstacionesEnSanFrancisco.start_station_name,\
                                                                       dfViajesEntreEstacionesEnSanFrancisco.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanFrancisco.end_station_long,\
                                                                       dfViajesEntreEstacionesEnSanFrancisco.end_station_name,\
                                                                       dfViajesEntreEstacionesEnSanFrancisco.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresSanFrancisco(cant), weight=3, opacity=1).add_to(viajesEntreEstacionesEnSanFranciscoMap)
viajesEntreEstacionesEnSanFranciscoMap

In [ ]:
### Grafico con n estaciones mas activas

In [ ]:
n = 20

In [ ]:
cotaInferiorSanFrancisco = dfViajesEntreEstacionesEnSanFrancisco.cantidad.nlargest (n).min()

In [ ]:
dfViajesEntreEstacionesEnSanFranciscoTopN = dfViajesEntreEstacionesEnSanFrancisco[dfViajesEntreEstacionesEnSanFrancisco['cantidad'] \
                                                                        >= cotaInferiorSanFrancisco]
dfViajesEntreEstacionesEnSanFranciscoTopN

In [ ]:
topViajesEntreEstacionesEnSanFranciscoMap = folium.Map(location=[latSanFranciscoPromedio, longSanFranciscoPromedio], tiles = "Stamen Terrain", zoom_start=14)
topViajesEntreEstacionesEnSanFranciscoMap

In [ ]:
dfViajesEntreEstacionesEnSanFranciscoTopN.cantidad.describe()

In [ ]:
coloresSanFranciscoTopN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnSanFrancisco, index=[3033, 3435, 3574, 4291, maxCantidadDeViajesEnSanFrancisco],
                       caption='step')
coloresSanFranciscoTopN.caption = 'Cantidad de viajes entre estaciones en San Francisco'

coloresSanFranciscoTopN2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnSanFrancisco)

coloresSanFranciscoTopN

In [ ]:
topViajesEntreEstacionesEnSanFranciscoMap.add_child(coloresSanFranciscoTopN)

In [ ]:
clusterDeUbicacionesSanFrancisco = folium.MarkerCluster().add_to(topViajesEntreEstacionesEnSanFranciscoMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnSanFrancisco.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnSanFrancisco.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnSanFrancisco.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesSanFrancisco)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnSanFranciscoTopN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoTopN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoTopN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoTopN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoTopN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoTopN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoTopN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresSanFranciscoTopN(cant), weight=5, opacity=1).add_to(topViajesEntreEstacionesEnSanFranciscoMap)
topViajesEntreEstacionesEnSanFranciscoMap

### Grafico con n estaciones menos activas

In [ ]:
n = 20

In [ ]:
cotaSuperiorSanFrancisco = dfViajesEntreEstacionesEnSanFrancisco.cantidad.nsmallest(n).max()

In [ ]:
dfViajesEntreEstacionesEnSanFranciscoBottomN = dfViajesEntreEstacionesEnSanFrancisco[dfViajesEntreEstacionesEnSanFrancisco['cantidad'] \
                                                                        <= cotaSuperiorSanFrancisco]
dfViajesEntreEstacionesEnSanFranciscoBottomN

In [ ]:
bottomViajesEntreEstacionesEnSanFranciscoMap = folium.Map(location=[latSanFranciscoPromedio, longSanFranciscoPromedio], tiles = "Stamen Terrain", zoom_start=14)
bottomViajesEntreEstacionesEnSanFranciscoMap

In [ ]:
dfViajesEntreEstacionesEnSanFranciscoBottomN.cantidad.describe()

In [ ]:
coloresSanFranciscoBottomN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=30, index=[9, 15, 21, 26, 30],
                       caption='step')
coloresSanFranciscoBottomN.caption = 'Cantidad de viajes entre estaciones con menos viajes entre ellas en San Francisco'

coloresSanFranciscoBottomN2 = cm.linear.YlGn.scale(0, 13)

coloresSanFranciscoBottomN

In [ ]:
bottomViajesEntreEstacionesEnSanFranciscoMap.add_child(coloresSanFranciscoBottomN)

In [ ]:
clusterDeUbicacionesSanFrancisco = folium.MarkerCluster().add_to(bottomViajesEntreEstacionesEnSanFranciscoMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnSanFrancisco.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnSanFrancisco.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnSanFrancisco.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesSanFrancisco)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnSanFranciscoBottomN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoBottomN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoBottomN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoBottomN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoBottomN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoBottomN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnSanFranciscoBottomN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresSanFranciscoBottomN(cant), weight=5, opacity=1).add_to(bottomViajesEntreEstacionesEnSanFranciscoMap)
bottomViajesEntreEstacionesEnSanFranciscoMap

# Mapa con viajes en Redwood City

In [ ]:
dfViajesEntreEstacionesEnRedwoodCity = (dfViajesEntreEstaciones[dfViajesEntreEstaciones.start_city == 'Redwood City']).drop('start_city', 1)
dfViajesEntreEstacionesEnRedwoodCity = (dfViajesEntreEstacionesEnRedwoodCity[dfViajesEntreEstacionesEnRedwoodCity.end_city == 'Redwood City']).drop('end_city', 1)
dfViajesEntreEstacionesEnRedwoodCity

In [ ]:
latRedwoodCityPromedio = dfViajesEntreEstacionesEnRedwoodCity.start_station_lat.mean()
longRedwoodCityPromedio = dfViajesEntreEstacionesEnRedwoodCity.start_station_long.mean()

In [ ]:
viajesEntreEstacionesEnRedwoodCityMap = folium.Map(location=[latRedwoodCityPromedio, longRedwoodCityPromedio], tiles = "Stamen Terrain", zoom_start=15)
viajesEntreEstacionesEnRedwoodCityMap

In [ ]:
dfViajesEntreEstacionesEnRedwoodCity.cantidad.describe()

In [ ]:
maxCantidadDeViajesEnRedwoodCity = dfViajesEntreEstacionesEnRedwoodCity.cantidad.max()

In [ ]:
coloresRedwoodCity = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnRedwoodCity, index=[1, 9, 25, 74, maxCantidadDeViajesEnRedwoodCity],
                       caption='step')
coloresRedwoodCity.caption = 'Cantidad de viajes entre estaciones en Redwood City'
coloresRedwoodCity
coloresRedwoodCity2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnRedwoodCity)

In [ ]:
viajesEntreEstacionesEnRedwoodCityMap.add_child(coloresRedwoodCity)

In [ ]:
clusterDeUbicacionesRedwoodCity = folium.MarkerCluster().add_to(viajesEntreEstacionesEnRedwoodCityMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnRedwoodCity.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnRedwoodCity.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnRedwoodCity.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesRedwoodCity)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnRedwoodCity.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnRedwoodCity.start_station_long,\
                                                                       dfViajesEntreEstacionesEnRedwoodCity.start_station_name,\
                                                                       dfViajesEntreEstacionesEnRedwoodCity.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnRedwoodCity.end_station_long,\
                                                                       dfViajesEntreEstacionesEnRedwoodCity.end_station_name,\
                                                                       dfViajesEntreEstacionesEnRedwoodCity.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresRedwoodCity(cant), weight=3, opacity=1).add_to(viajesEntreEstacionesEnRedwoodCityMap)
viajesEntreEstacionesEnRedwoodCityMap

### Grafico con n estaciones mas activas

In [ ]:
n = 20

In [ ]:
cotaInferiorRedwoodCity = dfViajesEntreEstacionesEnRedwoodCity.cantidad.nlargest (n).min()

In [ ]:
dfViajesEntreEstacionesEnRedwoodCityTopN = dfViajesEntreEstacionesEnRedwoodCity[dfViajesEntreEstacionesEnRedwoodCity['cantidad'] \
                                                                        >= cotaInferiorRedwoodCity]
dfViajesEntreEstacionesEnRedwoodCityTopN

In [ ]:
topViajesEntreEstacionesEnRedwoodCityMap = folium.Map(location=[latRedwoodCityPromedio, longRedwoodCityPromedio], tiles = "Stamen Terrain", zoom_start=14)
topViajesEntreEstacionesEnRedwoodCityMap

In [ ]:
dfViajesEntreEstacionesEnRedwoodCityTopN.cantidad.describe()

In [ ]:
coloresRedwoodCityTopN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnRedwoodCity, index=[38, 63, 113, 214, maxCantidadDeViajesEnSanJose],
                       caption='step')
coloresRedwoodCityTopN.caption = 'Cantidad de viajes entre estaciones en Redwood City'

coloresRedwoodCityTopN2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnRedwoodCity)

coloresRedwoodCityTopN

In [ ]:
topViajesEntreEstacionesEnRedwoodCityMap.add_child(coloresRedwoodCityTopN)

In [ ]:
clusterDeUbicacionesRedwoodCity = folium.MarkerCluster().add_to(topViajesEntreEstacionesEnRedwoodCityMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnRedwoodCity.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnRedwoodCity.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnRedwoodCity.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesRedwoodCity)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnRedwoodCityTopN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityTopN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityTopN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityTopN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityTopN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityTopN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityTopN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresRedwoodCityTopN(cant), weight=5, opacity=1).add_to(topViajesEntreEstacionesEnRedwoodCityMap)
topViajesEntreEstacionesEnRedwoodCityMap

### Grafico con n estaciones menos activas

In [ ]:
n = 20

In [ ]:
cotaSuperiorRedwoodCity = dfViajesEntreEstacionesEnRedwoodCity.cantidad.nsmallest(n).max()

In [ ]:
dfViajesEntreEstacionesEnRedwoodCityBottomN = dfViajesEntreEstacionesEnRedwoodCity[dfViajesEntreEstacionesEnRedwoodCity['cantidad'] \
                                                                        <= cotaSuperiorRedwoodCity]
dfViajesEntreEstacionesEnRedwoodCityBottomN

In [ ]:
bottomViajesEntreEstacionesEnRedwoodCityMap = folium.Map(location=[latRedwoodCityPromedio, longRedwoodCityPromedio], tiles = "Stamen Terrain", zoom_start=14)
bottomViajesEntreEstacionesEnRedwoodCityMap

In [ ]:
dfViajesEntreEstacionesEnRedwoodCityBottomN.cantidad.describe()

In [ ]:
coloresRedwoodCityBottomN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=17, index=[1, 4, 7, 11, 17],
                       caption='step')
coloresRedwoodCityBottomN.caption = 'Cantidad de viajes entre estaciones con menos viajes entre ellas en Redwood City'

coloresRedwoodCityBottomN2 = cm.linear.YlGn.scale(0, 13)

coloresRedwoodCityBottomN

In [ ]:
bottomViajesEntreEstacionesEnRedwoodCityMap.add_child(coloresRedwoodCityBottomN)

In [ ]:
clusterDeUbicacionesRedwoodCity = folium.MarkerCluster().add_to(bottomViajesEntreEstacionesEnRedwoodCityMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnRedwoodCity.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnRedwoodCity.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnRedwoodCity.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesRedwoodCity)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnRedwoodCityBottomN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityBottomN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityBottomN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityBottomN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityBottomN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityBottomN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnRedwoodCityBottomN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresRedwoodCityBottomN(cant), weight=5, opacity=1).add_to(bottomViajesEntreEstacionesEnRedwoodCityMap)
bottomViajesEntreEstacionesEnRedwoodCityMap

# Mapa con viajes en Palo Alto

In [ ]:
dfViajesEntreEstacionesEnPaloAlto = (dfViajesEntreEstaciones[dfViajesEntreEstaciones.start_city == 'Palo Alto']).drop('start_city', 1)
dfViajesEntreEstacionesEnPaloAlto = (dfViajesEntreEstacionesEnPaloAlto[dfViajesEntreEstacionesEnPaloAlto.end_city == 'Palo Alto']).drop('end_city', 1)
dfViajesEntreEstacionesEnPaloAlto

In [ ]:
latPaloAltoPromedio = dfViajesEntreEstacionesEnPaloAlto.start_station_lat.mean()
longPaloAltoPromedio = dfViajesEntreEstacionesEnPaloAlto.start_station_long.mean()

In [ ]:
viajesEntreEstacionesEnPaloAltoMap = folium.Map(location=[latPaloAltoPromedio, longPaloAltoPromedio], tiles = "Stamen Terrain", zoom_start=13)
viajesEntreEstacionesEnPaloAltoMap

In [ ]:
dfViajesEntreEstacionesEnPaloAlto.cantidad.describe()

In [ ]:
maxCantidadDeViajesEnPaloAlto = dfViajesEntreEstacionesEnPaloAlto.cantidad.max()

In [ ]:
coloresPaloAlto = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnPaloAlto, index=[56, 91, 169, 279, maxCantidadDeViajesEnPaloAlto],
                       caption='step')
coloresPaloAlto.caption = 'Cantidad de viajes entre estaciones en Palo Alto'
coloresPaloAlto
coloresPaloAlto2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnPaloAlto)

In [ ]:
viajesEntreEstacionesEnPaloAltoMap.add_child(coloresPaloAlto)

In [ ]:
clusterDeUbicacionesPaloAlto = folium.MarkerCluster().add_to(viajesEntreEstacionesEnPaloAltoMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnPaloAlto.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnPaloAlto.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnPaloAlto.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesPaloAlto)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnPaloAlto.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnPaloAlto.start_station_long,\
                                                                       dfViajesEntreEstacionesEnPaloAlto.start_station_name,\
                                                                       dfViajesEntreEstacionesEnPaloAlto.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnPaloAlto.end_station_long,\
                                                                       dfViajesEntreEstacionesEnPaloAlto.end_station_name,\
                                                                       dfViajesEntreEstacionesEnPaloAlto.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresPaloAlto(cant), weight=3, opacity=1).add_to(viajesEntreEstacionesEnPaloAltoMap)
viajesEntreEstacionesEnPaloAltoMap

### Grafico con n estaciones mas activas

In [ ]:
n = 20

In [ ]:
cotaInferiorPaloAlto = dfViajesEntreEstacionesEnPaloAlto.cantidad.nlargest (n).min()

In [ ]:
dfViajesEntreEstacionesEnPaloAltoTopN = dfViajesEntreEstacionesEnPaloAlto[dfViajesEntreEstacionesEnPaloAlto['cantidad'] \
                                                                        >= cotaInferiorPaloAlto]
dfViajesEntreEstacionesEnPaloAltoTopN

In [ ]:
topViajesEntreEstacionesEnPaloAltoMap = folium.Map(location=[latPaloAltoPromedio, longPaloAltoPromedio], tiles = "Stamen Terrain", zoom_start=14)
topViajesEntreEstacionesEnPaloAltoMap

In [ ]:
dfViajesEntreEstacionesEnPaloAltoTopN.cantidad.describe()

In [ ]:
coloresPaloAltoTopN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesEnPaloAlto, index=[90, 121, 234, 373, maxCantidadDeViajesEnPaloAlto],
                       caption='step')
coloresPaloAltoTopN.caption = 'Cantidad de viajes entre estaciones en Palo Alto'

coloresPaloAltoTopN2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesEnPaloAlto)

coloresPaloAltoTopN

In [ ]:
topViajesEntreEstacionesEnPaloAltoMap.add_child(coloresPaloAltoTopN)

In [ ]:
clusterDeUbicacionesPaloAlto = folium.MarkerCluster().add_to(topViajesEntreEstacionesEnPaloAltoMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnPaloAlto.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnPaloAlto.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnPaloAlto.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesPaloAlto)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnPaloAltoTopN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnPaloAltoTopN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnPaloAltoTopN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnPaloAltoTopN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnPaloAltoTopN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnPaloAltoTopN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnPaloAltoTopN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresPaloAltoTopN(cant), weight=5, opacity=1).add_to(topViajesEntreEstacionesEnPaloAltoMap)
topViajesEntreEstacionesEnPaloAltoMap

### Grafico con n estaciones menos activas

In [ ]:
n = 20

In [ ]:
cotaSuperiorPaloAlto = dfViajesEntreEstacionesEnPaloAlto.cantidad.nsmallest(n).max()

In [ ]:
dfViajesEntreEstacionesEnPaloAltoBottomN = dfViajesEntreEstacionesEnPaloAlto[dfViajesEntreEstacionesEnPaloAlto['cantidad'] \
                                                                        <= cotaSuperiorPaloAlto]
dfViajesEntreEstacionesEnPaloAltoBottomN

In [ ]:
bottomViajesEntreEstacionesEnPaloAltoMap = folium.Map(location=[latPaloAltoPromedio, longPaloAltoPromedio], tiles = "Stamen Terrain", zoom_start=14)
bottomViajesEntreEstacionesEnPaloAltoMap

In [ ]:
dfViajesEntreEstacionesEnPaloAltoBottomN.cantidad.describe()

In [ ]:
coloresPaloAltoBottomN = cm.StepColormap(['green', 'yellow', 'magenta', 'red'],
                       vmin=1, vmax=366, index=[56, 89, 118, 223, 366],
                       caption='step')
coloresPaloAltoBottomN.caption = 'Cantidad de viajes entre estaciones con menos viajes entre ellas en Palo Alto'

coloresPaloAltoBottomN2 = cm.linear.YlGn.scale(0, 366)

coloresPaloAltoBottomN

In [ ]:
bottomViajesEntreEstacionesEnPaloAltoMap.add_child(coloresPaloAltoBottomN)

In [ ]:
clusterDeUbicacionesPaloAlto = folium.MarkerCluster().add_to(bottomViajesEntreEstacionesEnPaloAltoMap)
for lat, lon, nombre in zip(dfViajesEntreEstacionesEnPaloAlto.start_station_lat.unique(), \
                            dfViajesEntreEstacionesEnPaloAlto.start_station_long.unique(), \
                            dfViajesEntreEstacionesEnPaloAlto.start_station_name.unique()):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicacionesPaloAlto)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesEnPaloAltoBottomN.start_station_lat,\
                                                                       dfViajesEntreEstacionesEnPaloAltoBottomN.start_station_long,\
                                                                       dfViajesEntreEstacionesEnPaloAltoBottomN.start_station_name,\
                                                                       dfViajesEntreEstacionesEnPaloAltoBottomN.end_station_lat,\
                                                                       dfViajesEntreEstacionesEnPaloAltoBottomN.end_station_long,\
                                                                       dfViajesEntreEstacionesEnPaloAltoBottomN.end_station_name,\
                                                                       dfViajesEntreEstacionesEnPaloAltoBottomN.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresPaloAltoBottomN(cant), weight=5, opacity=1).add_to(bottomViajesEntreEstacionesEnPaloAltoMap)
bottomViajesEntreEstacionesEnPaloAltoMap

# Viajes entre estaciones de distintas ciudades

In [ ]:
dfViajesEntreEstacionesDeDistinasCiudades = (dfViajesEntreEstaciones[dfViajesEntreEstaciones.start_city != dfViajesEntreEstaciones.end_city])
# dfViajesEntreEstacionesEnPaloAlto = (dfViajesEntreEstacionesEnPaloAlto[dfViajesEntreEstacionesEnPaloAlto.end_city == 'Palo Alto']).drop('end_city', 1)
dfViajesEntreEstacionesDeDistinasCiudades.sort_values(by='cantidad')

In [ ]:
latDeDistinasCiudadesPromedio = dfViajesEntreEstacionesDeDistinasCiudades.start_station_lat.mean()
longDeDistinasCiudadesPromedio = dfViajesEntreEstacionesDeDistinasCiudades.start_station_long.mean()

In [ ]:
viajesEntreEstacionesDeDistinasCiudadesMap = folium.Map(location=[latDeDistinasCiudadesPromedio, longDeDistinasCiudadesPromedio], tiles = "Stamen Terrain", zoom_start=10)
viajesEntreEstacionesDeDistinasCiudadesMap

In [ ]:
dfViajesEntreEstacionesDeDistinasCiudades.cantidad.describe()

In [ ]:
maxCantidadDeViajesDeDistinasCiudades = dfViajesEntreEstacionesDeDistinasCiudades.cantidad.max()

In [ ]:
coloresDeDistinasCiudades = cm.StepColormap(['green', 'magenta', 'red'],
                       vmin=1, vmax=maxCantidadDeViajesDeDistinasCiudades, index=[1, 2, 6, maxCantidadDeViajesDeDistinasCiudades],
                       caption='step')
coloresDeDistinasCiudades.caption = 'Cantidad de viajes entre estaciones de distinas ciudades'
coloresDeDistinasCiudades
coloresDeDistinasCiudades2 = cm.linear.YlGn.scale(0, maxCantidadDeViajesDeDistinasCiudades)

In [ ]:
viajesEntreEstacionesDeDistinasCiudadesMap.add_child(coloresDeDistinasCiudades)

In [ ]:
clusterDeUbicaciones = folium.MarkerCluster().add_to(viajesEntreEstacionesDeDistinasCiudadesMap)
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicaciones)

In [ ]:
for startLat, startLon, startName, endLat, endLon, endName, cant in zip(dfViajesEntreEstacionesDeDistinasCiudades.start_station_lat,\
                                                                       dfViajesEntreEstacionesDeDistinasCiudades.start_station_long,\
                                                                       dfViajesEntreEstacionesDeDistinasCiudades.start_station_name,\
                                                                       dfViajesEntreEstacionesDeDistinasCiudades.end_station_lat,\
                                                                       dfViajesEntreEstacionesDeDistinasCiudades.end_station_long,\
                                                                       dfViajesEntreEstacionesDeDistinasCiudades.end_station_name,\
                                                                       dfViajesEntreEstacionesDeDistinasCiudades.cantidad):
    folium.PolyLine(locations=[(startLat, startLon), (endLat, endLon)], color=coloresDeDistinasCiudades(cant), weight=3, opacity=1).add_to(viajesEntreEstacionesDeDistinasCiudadesMap)
viajesEntreEstacionesDeDistinasCiudadesMap

## Grafico de porcentaje de uso por estacion

In [ ]:
porcentajeDeViajesMap = folium.Map(location=[latPromedio, longPromedio], tiles = "Stamen Terrain")
porcentajeDeViajesMap

In [ ]:
clusterDeUbicaciones = folium.MarkerCluster().add_to(porcentajeDeViajesMap)
for lat, lon, nombre in zip(dfEstaciones.lat, dfEstaciones.long, dfEstaciones.name):
    folium.Marker(location=[lat,lon], popup=nombre).add_to(clusterDeUbicaciones)
    
porcentajeDeViajesMap

In [ ]:
minPercentageInUsagePercentagePerStation = dfusagePercentagePerStation.docks_usage_percentage.min()
maxPercentageInUsagePercentagePerStation = dfusagePercentagePerStation.docks_usage_percentage.max()
grupoDeUbicacionesConPorcentajesDeViajes = folium.FeatureGroup(name='Uso de las estaciones')
for lat, lon, nombre, porcentaje in zip(dfusagePercentagePerStation.lat, dfusagePercentagePerStation.long, \
                                        dfusagePercentagePerStation.name, dfusagePercentagePerStation.docks_usage_percentage):
    grupoDeUbicacionesConPorcentajesDeViajes.add_child(folium.CircleMarker(location=[lat,lon], popup=nombre, 
                                                     radius=(((porcentaje - minPercentageInUsagePercentagePerStation) * (80 - 1)) / (maxPercentageInUsagePercentagePerStation - minPercentageInUsagePercentagePerStation)) + 1, 
                                                     fill_color='#b3ffb3', color='#00ff00'))
    
porcentajeDeViajesMap.add_child(grupoDeUbicacionesConPorcentajesDeViajes)